# Animal Detection Pipeline
This notebook provides a streamlined pipeline to detect animals in images. The pipeline leverages the power of the ImageAI library and a custom-trained YOLOv3 model to achieve this task. Once the images are processed, the results, which include the type of animal detected, the confidence score, and bounding box coordinates, are stored in a pandas DataFrame.

## Pipeline Instructions:
### 1. File Upload:
* Before you begin, make sure you have the following files ready for upload:
    * `requirements_gpu.txt`: Contains the necessary Python packages.
    * `YOLO_yolov3_detection_config.json`: Configuration file for the YOLOv3 model.
    * `yolov3_YOLO_mAP-0.36626_epoch-24.pt`: The trained weights for the YOLOv3 model.

You can either manually upload these files using the provided code snippet or place them in your Google Drive and access them directly from there.

### 2. Package Installation:
Run the provided code cells to install the necessary packages. If you encounter any issues, consider manually uploading the `requirements_gpu.txt` file and retrying.

### 3. Dropbox Connection:
To fetch the images from Dropbox, you'll need to provide an API key. Follow the step-by-step guide in the notebook to obtain the API key from Dropbox.

### 4. Animal Detection:
Once everything is set up, you can use the provided functions to detect animals in the images:
* `detect_objects`: Detects animals in a given image.
* `process_image`: Processes an individual image using the detect_objects function.
* `process_images_in_directory`: Iterates over all images in a specified directory to detect animals.

For more details on each function, including their parameters and usage, refer to the "Guide to Animal Image Detection in Directories" section.

### 5. Results:
After processing the images, the results are stored in a pandas DataFrame. This DataFrame provides details of all detected animals, including:
* Image Name (relative to the `animal_images` directory)
* Animal Type
* Detection Confidence (Probability)
* Bounding Box Coordinates

You can view the top rows of the DataFrame using `df_results.head()` or the entire DataFrame with `df_results`.


By following the above instructions, you should be able to successfully utilize the animal detection pipeline.

### Files Upload
Please use the following code to upload the mentioned files:

* `requirements_gpu.txt`
* `YOLO_yolov3_detection_config.json`
* `yolov3_YOLO_mAP-0.36626_epoch-24.pt`

After running the code, you'll be prompted to select and upload the files from your device.

In [ ]:
from google.colab import files
uploaded = files.upload()

**Note:** Since the `yolov3_YOLO_mAP-0.36626_epoch-24.pt` file is quite large and may take a considerable amount of time to upload, there's an alternative. You can upload all the files to your Google Drive. Once the files are in your Google Drive, you can easily access them in your Colab notebook using the following code:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Install relevant packages

In [ ]:
!pip install dropbox --quiet
!pip install fastai --quiet
!pip install fastdup --quiet
!pip install imageai --upgrade --quiet
!pip cython
!pip pillow>=7.0.0
!pip numpy>=1.18.1
!pip opencv-python>=4.1.2
!pip torch>=1.9.0 --extra-index-url https://download.pytorch.org/whl/cu102
!pip torchvision>=0.10.0 --extra-index-url https://download.pytorch.org/whl/cu102
!pip pytest==7.1.3
!pip tqdm==4.64.1
!pip scipy>=1.7.3
!pip matplotlib>=3.4.3
!pip mock==4.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 5.5 MB/s eta 0:00:00


If you encounter issues executing the command `!pip install -r requirements_gpu.txt --quiet`, please consider manually uploading the requirements_gpu.txt file.

#### Import packages

In [ ]:
import dropbox
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tempfile
import os
import shutil
import fastdup
import cv2
import numpy as np
import torch

## Setup Connection to DropBox

In this section you will need to provide dropbox your api key in order to connect the images to this python notebook.

<u> How to get the API key? </u>

To get an API key (or access token) for Dropbox, you'll need to create an app in the Dropbox App Console. Here's a step-by-step guide to obtain the API key:



1.   **Log into Dropbox:**
    
    Go to [Dropbox](https://www.dropbox.com/home) and sign in to your account.
    

2.   **Access the Dropbox Developer Console:**
    Go to Dropbox and sign in to your account.
    Access the [Dropbox Developer Console](https://www.dropbox.com/developers/apps):



3. **Create a New App:**

* Click on the 'Create App' button.
Choose the following type of access:
Full dropbox: Access to all files and folders in your Dropbox.
* Name your app. This name has to be unique across Dropbox, so you might need to try a few times to find an available name.
* Click the "Create App" button.

4. **Configure Permissions** (especially if you chose Scoped Access in step 3):

* Once your app is created, you'll be taken to its settings page.

* Under the "OAuth 2" section, you can specify the permissions your app needs. For example, if your app needs to read and write files, you'd request the files.content.read and files.content.write permissions.
* After selecting the necessary permissions, click the "Submit" button at the bottom of the page to save your changes.

4. **Generate the Access Token:**
* Still on your app's settings page, find the "OAuth 2" section.
* Click the "Generate" button in the "Generated access token" subsection.
* Your token will appear. Copy this token and store it securely. Treat this token like a password – it provides full access to your app and any files/folders associated with the permissions you've set.

5. **Use the Access Token:**
* With this token, you can make API calls to Dropbox on behalf of the account associated with the token.

In [ ]:
api_key = "" # Add your DropBox API key
dbx_api_key = api_key
dbx = dropbox.Dropbox(dbx_api_key)

In [ ]:
def download_files_from_folder(dbx, dropbox_folder, local_folder):
    """
    Recursively downloads files from a Dropbox folder to a local directory.

    Args:
    - dbx (dropbox.Dropbox): Dropbox instance.
    - dropbox_folder (str): Path to the folder in Dropbox.
    - local_folder (str): Local directory to save the downloaded files.
    """
    try:
        entries = dbx.files_list_folder(dropbox_folder).entries
    except dropbox.exceptions.ApiError as e:
        print(f"Error listing contents of {dropbox_folder}: {e}")
        return

    for entry in entries:
        if isinstance(entry, dropbox.files.FileMetadata):
            # It's a file, so download it
            file_path = os.path.join(dropbox_folder, entry.name)
            local_path = os.path.join(local_folder, entry.name)
            try:
                dbx.files_download_to_file(local_path, file_path)
            except dropbox.exceptions.ApiError as e:
                print(f"Error downloading {file_path}: {e}")
        elif isinstance(entry, dropbox.files.FolderMetadata):
            # It's a folder, so create a corresponding local folder and recurse
            local_subfolder = os.path.join(local_folder, entry.name)
            if not os.path.exists(local_subfolder):
                os.makedirs(local_subfolder)
            download_files_from_folder(dbx, entry.path_lower, local_subfolder)

def download_animal_images(base_folder_path, local_image_dir="animal_images"):
    """
    Downloads all files from a Dropbox directory (and its subdirectories) to a local directory.

    Args:
    - base_folder_path (str): Base folder path in Dropbox.
    - local_image_dir (str): Local directory to save all downloaded images.
    """
    # Ensure the local directory exists
    if not os.path.exists(local_image_dir):
        os.makedirs(local_image_dir)

    download_files_from_folder(dbx, base_folder_path, local_image_dir)

To load your desired file, kindly modify the image_path variable to reflect the appropriate directory. For instance, if you intend to access the summer images of 2023, adjust images_path as follows:

`images_path = "/SUMMER 2023 animals only/".`

In [ ]:
images_path = "/SUMMER 2023 animals only/"
download_animal_images(images_path)

# Load trained ImageAI model for Animal Detection

In this section, we will load the pre-trained animal detection algorithm. This algorithm has been trained to identify animals in images. We will apply it to the images downloaded from images_path.

In [ ]:
from imageai.Detection.Custom import CustomObjectDetection
import re
import matplotlib.patches as patches

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()

Load the last checkpoint of the trained model to perform detection.

In [ ]:
# change the path with your files
yolo3_pt_file_path = "/content/drive/MyDrive/BGU Courses/data mining with miki project/yolov3_YOLO_mAP-0.36626_epoch-24.pt"
yolo3_json_file_path = "/content/drive/MyDrive/BGU Courses/data mining with miki project/YOLO_yolov3_detection_config.json"
detector.setModelPath(yolo3_pt_file_path)
detector.setJsonPath(yolo3_json_file_path)
detector.loadModel()

####Animal Detections Functions

In [ ]:
def detect_objects(detector, image_path, verbose=False, **kwargs):
    # Handle directory creation for extracted objects
    output_image_path = kwargs.get('output_image_path', None)

    # Extract the directory name and filename from the image_path
    relative_path = os.path.relpath(image_path, start='animal_images')
    dir_name, file_name = os.path.split(relative_path)

    if kwargs.get('extract_detected_objects', False) and output_image_path:
        full_output_path = os.path.join(output_image_path, dir_name)

        if not os.path.exists(full_output_path):
            os.makedirs(full_output_path)

        # Adjust output image path for extracted objects and original image
        kwargs['output_image_path'] = os.path.join(full_output_path, file_name)

    detections = detector.detectObjectsFromImage(
        input_image=image_path,
        **kwargs
    )

    # If the return type is a tuple, check the first element's type
    if isinstance(detections, tuple):
        # If the first element is a numpy array, no objects were detected
        if isinstance(detections[0], np.ndarray):
            if verbose:
                print(f"No objects detected in {image_path}")
            return []
        else:
            detections = detections[0]

    return detections

def process_image(detector, image_path, verbose=False, **kwargs):
    if verbose:
        print(f"Processing image: {image_path}")

    detections = detect_objects(detector, image_path, verbose=verbose, **kwargs)
    results = []

    # Append detections to results list
    for detection in detections:
        # Print detected object details
        if verbose:
            print(f"Detected {detection['name']} with {detection['percentage_probability']:.2f}% confidence.")
            print(f"Bounding box coordinates: {detection['box_points']}")

        relative_path = os.path.relpath(image_path, start='animal_images')
        results.append({
            'image_name': relative_path,
            'animal': detection["name"],
            'probability': detection["percentage_probability"],
            'box_points': detection["box_points"]
        })

    return results

def process_images_in_directory(detector, directory="animal_images", verbose=False, **kwargs):
    results = []

    # Iterate through all files and subdirectories in the directory
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(('.png', '.jpg', '.jpeg', 'JPG')):  # Add other image formats if needed
                filepath = os.path.join(root, filename)
                if verbose:
                    print(f"Found image file: {filepath}")
                image_results = process_image(detector, filepath, verbose=verbose, **kwargs)
                results.extend(image_results)

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df


def process_images_in_directory_10_images(detector, directory="animal_images", verbose=False, **kwargs):
    results = []
    image_count = 0  # Counter for the number of images processed

    # Iterate through all files and subdirectories in the directory
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if image_count >= 10:  # Break the loop after 10 images
                break

            if filename.endswith(('.png', '.jpg', '.jpeg', 'JPG')):  # Add other image formats if needed
                filepath = os.path.join(root, filename)
                if verbose:
                    print(f"Found image file: {filepath}")
                image_results = process_image(detector, filepath, verbose=verbose, **kwargs)
                results.extend(image_results)
                image_count += 1

        if image_count >= 10:  # Break the outer loop as well
            break

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df

def crop_and_save_image(image_path, box_points, output_path):
    """
    Crop an image based on bounding box coordinates and save to the given path.
    """
    image = cv2.imread(image_path)
    y1, x1, y2, x2 = box_points  # Assuming box_points is [y1, x1, y2, x2]
    cropped_image = image[x1:x2, y1:y2]
    cv2.imwrite(output_path, cropped_image)

def process_and_save_cropped_images(detector, directory="animal_images", output_directory="cropped_images", verbose=False, **kwargs):
    # Get detections
    df = process_images_in_directory(detector, directory, verbose=verbose, **kwargs)

    # Process and save each detected object
    for _, row in df.iterrows():
        original_image_path = os.path.join(directory, row['image_name'])
        # Extracting the directory (e.g., "2030") from the image path
        image_dir = os.path.dirname(row['image_name'])

        # Define the animal directory path
        animal_dir = os.path.join(output_directory, row['animal'], image_dir)
        if not os.path.exists(animal_dir):
            os.makedirs(animal_dir)

        # Now, the output path will have the format: Animal/2030/IMG_0137.JPG
        output_image_path = os.path.join(animal_dir, os.path.basename(row['image_name']))

        crop_and_save_image(original_image_path, row['box_points'], output_image_path)

        if verbose:
            print(f"Saved cropped image for {row['animal']} to {output_image_path}")

    return df  # You can return the DataFrame if needed or any other summary info



### Guide to Animal Image Detection in Directories

These functions allow users to process images stored in a directory (and its subdirectories) to detect animals using a trained detector. Detected animals and their details are saved to a pandas DataFrame.

Functions:

1. **detect_objects**:
* Detects animals in a given image.
* Parameters:
    * **image_path:** Path of the image to process.
    * **verbose** (default: False): Prints additional information about the processing if set to True.
    * **extract_detected_objects** (default: False): Extracts detected objects as separate images if set to True.
    * **minimum_percentage_probability** (default: 50): Sets the minimum probability threshold for a detection to be considered.
    * **nms_treshold** (default: 0.1): Non-maximum suppression threshold to filter weak overlapping bounding boxes.
    * **output_image_path** (optional): Path to save an image with highlighted detections.
    * **display_percentage_probability** (default: True): Annotates saved image with detection probabilities.
    * **display_object_name** (default: True): Annotates saved image with detected object names.
2. **process_image**:

* Processes an individual image to detect animals using the `detect_objects` function.
* Parameters:
    * **image_path**: Path of the image to process.
    * **verbose** (default: False): Prints additional information about the processing if set to True.
Any other parameters that detect_objects accepts can be passed using **kwargs.

3. process_images_in_directory:

* Iterates over all images in a specified directory and its subdirectories to detect animals.
* Results are saved to a pandas DataFrame.
* **Parameters**:
    * **directory** (default: "animal_images"): Directory containing the images to process.
    * **verbose** (default: False): Prints additional information about the processing if set to True.
    * Any other parameters that detect_objects accepts can be passed using **kwargs.

### **Important Note**
If you don't want to save the images of the detected animals, please follow these steps:
* Change the parameter `extract_detected_objects` to `False`.
* Remove the parameter `output_image_path`.



**Example Usage:**

In [ ]:
# Process images in the 'animal_images' directory
df_results = process_and_save_cropped_images(
    detector,
    directory="animal_images",
    extract_detected_objects=False,
    minimum_percentage_probability=50,
    verbose=True
)

# The resulting DataFrame, df_results, contains details of all detected animals.
df_results.head()

Found image file: animal_images/2030/IMG_0137.JPG
Processing image: animal_images/2030/IMG_0137.JPG
Detected Fox with 66.54% confidence.
Bounding box coordinates: [1229, 175, 2991, 1158]
Detected Fox with 87.98% confidence.
Bounding box coordinates: [659, 0, 3521, 1526]
Found image file: animal_images/2030/IMG_0132.JPG
Processing image: animal_images/2030/IMG_0132.JPG
Detected Fox with 72.63% confidence.
Bounding box coordinates: [158, 610, 527, 1026]
Found image file: animal_images/2030/IMG_0136.JPG
Processing image: animal_images/2030/IMG_0136.JPG
Detected Fox with 67.22% confidence.
Bounding box coordinates: [1907, 25, 3329, 922]
Detected Fox with 58.81% confidence.
Bounding box coordinates: [1235, 0, 3712, 1419]
Found image file: animal_images/2030/IMG_0127.JPG
Processing image: animal_images/2030/IMG_0127.JPG
Found image file: animal_images/2030/IMG_0025.JPG
Processing image: animal_images/2030/IMG_0025.JPG
Detected Jackal with 60.64% confidence.
Bounding box coordinates: [101, 35

,image_name,animal,probability,box_points
0,2030/IMG_0137.JPG,Fox,66.54,"[1229, 175, 2991, 1158]"
1,2030/IMG_0137.JPG,Fox,87.98,"[659, 0, 3521, 1526]"
2,2030/IMG_0132.JPG,Fox,72.63,"[158, 610, 527, 1026]"
3,2030/IMG_0136.JPG,Fox,67.22,"[1907, 25, 3329, 922]"
4,2030/IMG_0136.JPG,Fox,58.81,"[1235, 0, 3712, 1419]"


This example demonstrates how to process all images in the `"animal_images"` directory, extract detected objects into separate images, and save them in the `"output_images"` directory. The resulting DataFrame df_results contains details of all detected animals, including their names, probabilities, and bounding box coordinates. Setting `verbose=True` will print out additional information during the processing.

### Download the detected Animals Table
After processing the images and detecting the animals, the results are stored in a pandas DataFrame. This section allows you to download the results as a CSV file named `detected_animals.csv` to your local machine for further analysis or reporting.

In [ ]:
# Save the DataFrame to a CSV file
csv_filename = "detected_animals.csv"
df_results.to_csv(csv_filename, index=False)

# Trigger a download prompt for the user
from google.colab import files
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>